In [28]:
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress
from scipy.stats import t
import sympy as sp

def sqrt_sumsq(arr):
    place_holder = []
    for i in arr:
        place_holder.append(i**2)
    return np.sqrt(np.sum(place_holder))

def Precision_Uncertainty(arr):
    
    #Precision Uncertainty Calculation Initialization
    Ns = len(arr)
    dof = Ns - 1  # degrees of freedom for mean
    t_crit = t.ppf(0.975, dof) #0.975 for two-tails student-t
    
    Prec_Unc = t_crit * np.nanstd(arr, ddof=1) / np.sqrt(Ns)
    return Prec_Unc

def Bias_Uncertainty(Acc, Res):
    
    Bias_Unc = sqrt_sumsq([Acc, Res/2])
    
    return Bias_Unc

def UPE_dPt(
    dPs_val, U_dPs_val,
    mass_val, U_mass_val,
    vol_val,  U_vol_val,
    dh_val,   U_dh_val,
    m_dot_val, U_mdot_val,
    A1_val, A2_val,
    g_val=9.807
):
    # 1) Symbolic setup with mass and volume
    dPs, mass, vol, g, dh, m_dot, A1, A2 = sp.symbols('dPs mass vol g dh m_dot A1 A2', real=True)
    U_dPs, U_mass, U_vol, U_dh, U_mdot = sp.symbols('U_dPs U_mass U_vol U_dh U_mdot', real=True, positive=True)
    
    # 2) Define rho = mass/vol and the formula for dPt
    rho = mass/vol
    dPt = (
        dPs
        + rho * g * dh
        + sp.Rational(1,2) * rho * ((m_dot/(rho*A2))**2 - (m_dot/(rho*A1))**2)
    )
    
    # 3) Compute symbolic partial derivatives w.r.t variables
    vars_sym = (dPs, mass, vol, dh, m_dot)
    partials = [sp.diff(dPt, var) for var in vars_sym]
    
    # 4) Build combined uncertainty U_dPt
    U_vars = (U_dPs, U_mass, U_vol, U_dh, U_mdot)
    U_dPt = sp.sqrt(sum((partials[i] * U_vars[i])**2 for i in range(len(vars_sym))))
    U_dPt_simplified = sp.simplify(U_dPt)
    
    # 5) Lambdify for numeric evaluation
    f_dPt      = sp.lambdify((dPs, mass, vol, g, dh, m_dot, A1, A2), dPt, 'numpy')
    f_partials = [sp.lambdify((dPs, mass, vol, g, dh, m_dot, A1, A2), pd, 'numpy') for pd in partials]

    # nominal result
    dPt_val = f_dPt(dPs_val, mass_val, vol_val, g_val, dh_val, m_dot_val, A1_val, A2_val)
    
    # compute gradients
    grads = np.array([f(dPs_val, mass_val, vol_val, g_val, dh_val, m_dot_val, A1_val, A2_val)
                      for f in f_partials], dtype=float)
    
    # uncertainties array
    U_vals = np.array([U_dPs_val, U_mass_val, U_vol_val, U_dh_val, U_mdot_val], dtype=float)
    
    # RSS propagation
    U_dPt_val = np.sqrt(np.sum((grads * U_vals)**2))
    return dPt_val, U_dPt_val, grads


def process_file(file_path):
    font_legend = {'family': 'TH Sarabun New',
                   'weight': 'bold',
                   'style': 'normal',
                   'size': 18}
    font_axis = {'family': 'TH Sarabun New',
                 'weight': 'bold',
                 'style': 'normal',
                 'size': 20}

    base_name  = os.path.splitext(os.path.basename(file_path))[0]
    parent_dir = os.path.dirname(file_path)
    output_dir = os.path.join(parent_dir, f"{base_name} plots")
    os.makedirs(output_dir, exist_ok=True)

    # Read CSV, skip first two summary rows, use third row as header
    df = pd.read_csv(file_path, delimiter=',', skiprows=2, header=0)
    x  = df['Run Time [s]']

    # 1) Mass Flow Rate vs Run Time (full) — raw data only, no curve fit
    plt.figure()
    plt.plot(x, df['Weight [Kg]'], label='Data')
    plt.xlabel('Run Time [s]', fontdict=font_axis)
    plt.ylabel('Weight [Kg]', fontdict=font_axis)
    plt.title('Mass Flow Rate vs Run Time', fontdict=font_axis)
    plt.legend(prop=font_legend)
    plt.ylim(bottom=0)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f"{base_name}_Mass_Flow_Rate_full_plot.png"))
    plt.close()

    # ... (Differential Pressure, Rotational Speed, Torque full-plot blocks unchanged) ...

    # --- FILTERED PLOTS (20 ≤ Weight ≤ 30 Kg) ---
    filtered = df[(df['Weight [Kg]'] >= 20) & (df['Weight [Kg]'] <= 30)]
    if not filtered.empty:
        
        frt = filtered['Run Time [s]'].iloc[-1] - filtered['Run Time [s]'].iloc[0]

        # Mass Flow Rate (filtered fit) + capture its standard error
        xf, yf = filtered['Run Time [s]'], filtered['Weight [Kg]']
        slope_f, intercept_f, r_f, _, stderr_f = linregress(xf, yf)
        # convert 1σ stderr into 95% CI half-width
        dof    = len(xf) - 2
        t_crit = t.ppf(0.975, dof)
        stderr_f *= t_crit
        r2_f = r_f**2
        max_mass = max(yf)
        min_mass = min(yf)
        plt.figure()
        plt.plot(xf, yf, label='Filtered')
        plt.plot(xf, slope_f*xf + intercept_f,
                 label=f'Fit: y={slope_f:.4f}x+{intercept_f:.4f}\nR²={r2_f:.4f}')
        plt.xlabel('Run Time [s]', fontdict=font_axis)
        plt.ylabel('Weight [Kg]', fontdict=font_axis)
        plt.title('Mass Flow Rate vs Run Time (Filtered)', fontdict=font_axis)
        plt.text(0.05, 0.95, f'Filtered Run Time: {frt:.2f} s',
                 transform=plt.gca().transAxes, verticalalignment='top', fontdict=font_axis)
        plt.legend(prop=font_legend); plt.ylim(bottom=0)
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f"{base_name}_Mass_Flow_Rate_filtered_plot.png"))
        plt.close()

        # Differential Pressure stats + plot
        dp     = filtered['Differential Pressure [Bar]']
        avg_dp = dp.mean()
        std_dp = 2 * dp.std(ddof=1)
        plt.figure()
        plt.plot(xf, dp)
        plt.xlabel('Run Time [s]', fontdict=font_axis)
        plt.ylabel('Differential Pressure [Bar]', fontdict=font_axis)
        plt.title('Differential Pressure vs Run Time (Filtered)', fontdict=font_axis)
        plt.text(0.05, 0.95,
                 f'Filtered Run Time: {frt:.2f} s\n'
                 f'Avg: {avg_dp:.4f} Bar\n'
                 f'SD:  {std_dp:.4f}',
                 transform=plt.gca().transAxes, verticalalignment='top', fontdict=font_axis)
        plt.ylim([0,3])
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f"{base_name}_Differential_Pressure_filtered_plot.png"))
        plt.close()

        # Rotational Speed stats + plot
        rs     = filtered['Rotational Speed [RPM]']
        avg_rs = rs.mean()
        std_rs = 2 * rs.std(ddof=1)
        plt.figure()
        plt.plot(xf, rs)
        plt.xlabel('Run Time [s]', fontdict=font_axis)
        plt.ylabel('Rotational Speed [RPM]', fontdict=font_axis)
        plt.title('Rotational Speed vs Run Time (Filtered)', fontdict=font_axis)
        plt.text(0.05, 0.95,
                 f'Filtered Run Time: {frt:.2f} s\n'
                 f'Avg: {avg_rs:.4f} RPM\n'
                 f'SD:  {std_rs:.4f}',
                 transform=plt.gca().transAxes, verticalalignment='top', fontdict=font_axis)
        plt.ylim([0,8500])
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f"{base_name}_Rotational_Speed_filtered_plot.png"))
        plt.close()

        # Torque stats + plot
        tq     = filtered['Torque [N-m]']
        avg_tq = tq.mean()
        std_tq = 2 * tq.std(ddof=1)
        plt.figure()
        plt.plot(xf, tq)
        plt.xlabel('Run Time [s]', fontdict=font_axis)
        plt.ylabel('Torque [N-m]', fontdict=font_axis)
        plt.title('Torque vs Run Time (Filtered)', fontdict=font_axis)
        plt.text(0.05, 0.95,
                 f'Filtered Run Time: {frt:.2f} s\n'
                 f'Avg: {avg_tq:.4f} N-m\n'
                 f'SD:  {std_tq:.4f}',
                 transform=plt.gca().transAxes, verticalalignment='top', fontdict=font_axis)
        plt.ylim([0,0.5])
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f"{base_name}_Torque_filtered_plot.png"))
        plt.close()
        
    else:
        slope_f = stderr_f = np.nan
        avg_dp = avg_rs = avg_tq = std_dp = std_rs = std_tq = np.nan
        print(f"{base_name}: no data in weight range 20–30 Kg")

    # ---- RETURN original metrics PLUS S.E. of slope ----
    return {
        'file':                base_name,
        'mass_flow_rate':      slope_f,
        'se_mass_flow_rate':   stderr_f,     # added standard error
        'mean_dp':             avg_dp,
        'mean_rs':             avg_rs,
        'mean_torque':         avg_tq,
        'std_dp':              std_dp,
        'std_rs':              std_rs,
        'std_tq':              std_tq,
        'run_time':            frt,
        'max_mass':            max_mass,
        'min_mass':            min_mass
    }


def main(csv_dir):
    files = glob.glob(os.path.join(csv_dir, '*.csv'))
    if not files:
        print(f"No CSV files found in {csv_dir}")
        return

    summaries = [process_file(f) for f in files]

    # ARRAYS of the PER-FILE time-averaged MEANS (unchanged)
    mfr_arr = np.array([s['mass_flow_rate'] for s in summaries])
    dp_arr  = np.array([s['mean_dp']        for s in summaries])
    rs_arr  = np.array([s['mean_rs']        for s in summaries])
    tq_arr  = np.array([s['mean_torque']    for s in summaries])
    run_time_arr = np.array([s['run_time']    for s in summaries])
    max_mass_arr = np.array([s['max_mass']    for s in summaries])
    min_mass_arr = np.array([s['min_mass']    for s in summaries])
    
    # ARRAYS of the PER-FILE SDs (unchanged)
    sd_dp_arr = np.array([s['std_dp'] for s in summaries])
    sd_rs_arr = np.array([s['std_rs'] for s in summaries])
    sd_tq_arr = np.array([s['std_tq'] for s in summaries])

    # ARRAY of the PER-FILE S.E.
    se_arr = np.array([s['se_mass_flow_rate'] for s in summaries])

    # GRAND MEANS & SDs of the MEANS (unchanged)
    grand_means = {
        'mfr_mean':  np.nanmean(mfr_arr), 'mfr_sd':  np.nanstd(mfr_arr, ddof=1),
        'dp_mean':   np.nanmean(dp_arr),  'dp_sd':   np.nanstd(dp_arr,  ddof=1),
        'rs_mean':   np.nanmean(rs_arr),  'rs_sd':   np.nanstd(rs_arr,  ddof=1),
        'tq_mean':   np.nanmean(tq_arr),  'tq_sd':   np.nanstd(tq_arr,  ddof=1),
        'run_time_mean':   np.nanmean(run_time_arr),  'run_time_sd':   np.nanstd(run_time_arr,  ddof=1),
        'max_mass_mean':   np.nanmean(max_mass_arr),  'max_mass_sd':   np.nanstd(max_mass_arr,  ddof=1),
        'min_mass_mean':   np.nanmean(min_mass_arr),  'min_mass_sd':   np.nanstd(min_mass_arr,  ddof=1),
    } #Single Number

    # ARITHMETIC AVERAGE OF THE SDs (unchanged)
    avg_sd = {
        'dp_bar': np.nanmean(sd_dp_arr),
        'rs_bar': np.nanmean(sd_rs_arr),
        'tq_bar': np.nanmean(sd_tq_arr),
    }

    # AVERAGE STANDARD ERROR OF MASS FLOW RATE (added)
    avg_se = np.nanmean(se_arr)
    
    grand_pu = {
        'mfr_pu':  Precision_Uncertainty(mfr_arr),
        'dp_pu':   Precision_Uncertainty(dp_arr),
        'rs_pu':   Precision_Uncertainty(rs_arr),
        'tq_pu':   Precision_Uncertainty(tq_arr)
    } #Single Number
    
    #Bias Uncertainty Code Area
    Sensor_Accuracy_Dict = {
        'mass_acc': 0.01, 'mass_res': 10 / 2**16 * 5, #kg
        'time_acc': 50e-6 * 0.05, 'time_res': 50e-9, #sec
        'dp_acc': 0.0075, 'dp_res': 10 / 2**16 * 0.3, #bar, res = 10V / 16-bit * 0.3 Bar/V, acc = 0.25%FS
        'tq_acc': 0.01, 'tq_res': 10 / 2**16 * 0.1, #N-m, res = 10V / 16-bit * 0.1 N-m/V
        'rs_acc': 0, 'rs_res': 0 #RPM   
    } #Single Number
    mass_bu = Bias_Uncertainty(Sensor_Accuracy_Dict['mass_acc'], Sensor_Accuracy_Dict['mass_res'])
    time_bu = Bias_Uncertainty(Sensor_Accuracy_Dict['time_acc'], Sensor_Accuracy_Dict['time_res'])
    dp_bu = Bias_Uncertainty(Sensor_Accuracy_Dict['dp_acc'], Sensor_Accuracy_Dict['dp_res'])
    rs_bu = Bias_Uncertainty(Sensor_Accuracy_Dict['rs_acc'], Sensor_Accuracy_Dict['rs_res'])
    tq_bu = Bias_Uncertainty(Sensor_Accuracy_Dict['tq_acc'], Sensor_Accuracy_Dict['tq_res'])
    
    #UPE for Mass Flow Rate from U_mass and U_time
    mfr_bu = np.sqrt((mass_bu/grand_means['run_time_mean'])**2 + ((grand_means['max_mass_mean'] - grand_means['min_mass_mean'])/grand_means['run_time_mean']**2*time_bu)**2)


    
    #Final Sum Uncertainty
    mfr_uncertainty = sqrt_sumsq([mfr_bu, grand_pu['mfr_pu'], avg_se])
    dp_uncertainty = sqrt_sumsq([dp_bu, grand_pu['dp_pu'], avg_sd['dp_bar']])
    rs_uncertainty = sqrt_sumsq([rs_bu, grand_pu['rs_pu'], avg_sd['rs_bar']])
    tq_uncertainty = sqrt_sumsq([tq_bu, grand_pu['tq_pu'], avg_sd['tq_bar']])
    
    #Derived Quantities Calculation : Total Pressure Rise | Shaft Power | Hydraulic Power | Efficiency
    
    # dPt Uncertainty Propagation Equation --> Using Bias only for other terms except dPs
    dPs, U_dPs = grand_means['dp_mean']*100000, dp_uncertainty*100000
    mass, U_mass = 1.17647, 0.001      # kg, uncertainty for Density Calculation
    vol,  U_vol  = 0.001, 0.3/1000000     # m^3, uncertainty for Density Calculation ±0.3 mL for 1000 mL Volumetric Flask
    dh,   U_dh   = 0.093, 0.02/1000      # m
    m_dot, U_mdot = grand_means['mfr_mean'], mfr_uncertainty   # kg/s
    A1, A2 = np.pi / 4 * 0.0188468**2, np.pi / 4 * 0.0138684**2         # m^2
    
    dPt_mean, dPt_uncertainty , _ = UPE_dPt(
        dPs, U_dPs, mass, U_mass, vol, U_vol,
        dh, U_dh, m_dot, U_mdot,
        A1, A2
    )
    
    dPt_mean = dPt_mean/100000
    dPt_uncertainty = dPt_uncertainty/100000
    
    #print(f"dPt    = {dPt_mean:.3f} Bar")
    #print(f"U_dPt  = {dPt_uncertainty:.3f} Bar")
    
    # Shaft Power Uncertainty Propagation Equation
    shaft_power_mean = grand_means['tq_mean'] * grand_means['rs_mean'] * 2 * np.pi / 60 # Watt
    shaft_power_uncertainty = sqrt_sumsq([grand_means['rs_mean'] * 2 * np.pi / 60 * tq_uncertainty , grand_means['tq_mean'] * 2 * np.pi / 60 * rs_uncertainty])
    
    # Density Uncertainty Propagation Equation
    density = mass / vol
    density_uncertainty = sqrt_sumsq([U_mass / vol, -1 * mass/vol**2 * U_vol])
    
    # Hydraulic Power Uncertainty Propagation Equation
    hydraulic_power_mean = dPt_mean * 100000 * grand_means['mfr_mean'] / (density)
    hydraulic_power_uncertainty = sqrt_sumsq([ grand_means['mfr_mean'] / density * dPt_uncertainty,   dPt_mean * 100000 / density * mfr_uncertainty, -1 * dPt_mean * 100000 *  grand_means['mfr_mean'] / density**2 * density_uncertainty])
    
    # Efficiency Uncertainty Propagation Equation
    efficiency_mean = hydraulic_power_mean / shaft_power_mean  
    efficiency_uncertainty = sqrt_sumsq([hydraulic_power_uncertainty /  shaft_power_mean, -1 * hydraulic_power_mean/shaft_power_mean**2 * shaft_power_uncertainty])
    
    
    
    # --- PRINT ---
    print("\nPer-file filtered MEANS:")
    for s in summaries:
        print(f" - {s['file']}: Mass Flow Rate={s['mass_flow_rate']:.4f}, "
              f"ΔPs={s['mean_dp']:.4f}, RPM={s['mean_rs']:.2f}, "
              f"Torque={s['mean_torque']:.4f}")

    print("\nGrand trimmed summary (of MEANS):")
    print(f" • Mass Flow Rate: mean={grand_means['mfr_mean']:.4f}, SD={grand_means['mfr_sd']:.4f}, Precision Uncertainty={grand_pu['mfr_pu']:.4f}, Bias Uncertainty={mfr_bu:.4f}")
    print(f" • ΔPs:             mean={grand_means['dp_mean']:.4f}, SD={grand_means['dp_sd']:.4f}, Precision Uncertainty={grand_pu['dp_pu']:.4f}, Bias Uncertainty={dp_bu:.4f}")
    print(f" • RPM:            mean={grand_means['rs_mean']:.2f}, SD={grand_means['rs_sd']:.2f}, Precision Uncertainty={grand_pu['rs_pu']:.4f}, Bias Uncertainty={rs_bu:.4f}")
    print(f" • Torque:         mean={grand_means['tq_mean']:.4f}, SD={grand_means['tq_sd']:.4f}, Precision Uncertainty={grand_pu['tq_pu']:.4f}, Bias Uncertainty={tq_bu:.4f}")


    print("\nPer-file filtered SDs:")
    for s in summaries:
        print(
            f" - {s['file']}: "
            f"SE Mass Flow Rate={s['se_mass_flow_rate']:.4f}, "
            f"2*SD ΔPs={s['std_dp']:.4f}, "
            f"2*SD RPM={s['std_rs']:.4f}, "
            f"2*SD Torque={s['std_tq']:.4f}"
        )

    print("\nAverage of SDs (SD̄):")
    print(f" • ΔPs 2*SD̄ = {avg_sd['dp_bar']:.4f}")
    print(f" • RPM 2*SD̄= {avg_sd['rs_bar']:.4f}")
    print(f" • Torque 2*SD̄ = {avg_sd['tq_bar']:.4f}")

    # Print average standard error
    print(f"\nAverage standard error of Mass Flow Rate = {avg_se:.4f}\n")

    # --- WRITE summary.txt ---
    folder = os.path.basename(csv_dir.rstrip(os.sep))
    summary_path = os.path.join(csv_dir, f"{folder} summary.txt")
    database_path = os.path.join(csv_dir, f"{folder} data.txt")
    with open(summary_path, 'w', encoding='utf-8') as f:
        
        f.write("\nTime-Averaged Values:\n")
        for s in summaries:
            f.write(f"{s['file']}: Mass Flow Rate={s['mass_flow_rate']:.4f}, "
                    f"ΔPs={s['mean_dp']:.4f}, RPM={s['mean_rs']:.2f}, "
                    f"Torque={s['mean_torque']:.4f}\n")
            
        f.write("\nTime-Averaged S.D. / S.E.:\n")
        
        for s in summaries:
            f.write(
                f"{s['file']}: "
                f"SE (95% CI) Mass Flow Rate={s['se_mass_flow_rate']:.4f}, "
                f"2*SD ΔPs={s['std_dp']:.4f}, "
                f"2*SD RPM={s['std_rs']:.4f}, "
                f"2*SD Torque={s['std_tq']:.4f}\n"
            )
        
        f.write("\nInstance Average of Means Value:\n")
        f.write(f"Mass Flow Rate:      mean={grand_means['mfr_mean']:.4f}, SD={grand_means['mfr_sd']:.4f}, Precision Uncertainty={grand_pu['mfr_pu']:.4f}, Bias Uncertainty={mfr_bu:.4f}\n")
        f.write(f"ΔPs:                 mean={grand_means['dp_mean']:.4f}, SD={grand_means['dp_sd']:.4f}, Precision Uncertainty={grand_pu['dp_pu']:.4f}, Bias Uncertainty={dp_bu:.4f}\n")
        f.write(f"RPM:                 mean={grand_means['rs_mean']:.2f}, SD={grand_means['rs_sd']:.2f}, Precision Uncertainty={grand_pu['rs_pu']:.4f}, Bias Uncertainty={rs_bu:.4f}\n")
        f.write(f"Torque:              mean={grand_means['tq_mean']:.4f}, SD={grand_means['tq_sd']:.4f}, Precision Uncertainty={grand_pu['tq_pu']:.4f}, Bias Uncertainty={tq_bu:.4f}\n\n")
        f.write(f"ΔPt:                 mean={dPt_mean:.4f} Bar\n")
        f.write(f"Shaft Power:         mean={shaft_power_mean:.4f} W\n")
        f.write(f"Hydraulic Power:     mean={hydraulic_power_mean:.4f} W\n")
        f.write(f"Efficiency:          mean={efficiency_mean*100:.2f}%\n")
        
        f.write("\nInstance Average of Time-Averaged SDs (SD̄):\n")
        f.write(f"S.E. (95% CI) of Mass Flow Rate = {avg_se:.4f}\n")
        f.write(f"ΔPs 2*SD̄ = {avg_sd['dp_bar']:.4f}\n")
        f.write(f"RPM 2*SD̄= {avg_sd['rs_bar']:.4f}\n")
        f.write(f"Torque 2*SD̄ = {avg_sd['tq_bar']:.4f}\n")

        f.write("\nFinal Uncertainty:\n")
        f.write(f"Uncertainty of Mass Flow Rate = {mfr_uncertainty:.4f} Kg/s\n")
        f.write(f"Uncertainty of ΔPs (RAW) = {dp_uncertainty:.4f} Bar\n")
        f.write(f"Uncertainty of RPM = {rs_uncertainty:.4f} RPM\n")
        f.write(f"Uncertainty of Torque = {tq_uncertainty:.4f} N-m\n\n")
        f.write(f"Uncertainty of ΔPt = {dPt_uncertainty:.4f} Bar\n")
        f.write(f"Uncertainty of Shaft Power = {shaft_power_uncertainty:.4f} W\n")
        f.write(f"Uncertainty of Hydraulic Power = {hydraulic_power_uncertainty:.4f} W\n")
        f.write(f"Uncertainty of Efficiency = {efficiency_uncertainty*100:.4f} %\n")

    print(f"Summary written to {summary_path}")
    
    
    with open(database_path, 'w', encoding='utf-8') as f:
        
        f.write("\nTime-Averaged Values:\n")
        for s in summaries:
            f.write(f"{s['file']}: Mass Flow Rate={s['mass_flow_rate']:.4f}, "
                    f"ΔPs={s['mean_dp']:.4f}, RPM={s['mean_rs']:.2f}, "
                    f"Torque={s['mean_torque']:.4f}\n")
            
        f.write("\nTime-Averaged S.D. / S.E.:\n")
        
        for s in summaries:
            f.write(
                f"{s['file']}: "
                f"SE (95% CI) Mass Flow Rate={s['se_mass_flow_rate']:.4f}, "
                f"2*SD ΔPs={s['std_dp']:.4f}, "
                f"2*SD RPM={s['std_rs']:.4f}, "
                f"2*SD Torque={s['std_tq']:.4f}\n"
            )
        
        f.write("\nInstance Average of Means Value:\n")
        f.write(f"Mass Flow Rate: mean={grand_means['mfr_mean']:.4f}, SD={grand_means['mfr_sd']:.4f}, Precision Uncertainty={grand_pu['mfr_pu']:.4f}, Bias Uncertainty={mfr_bu:.4f}\n")
        f.write(f"ΔPs:            mean={grand_means['dp_mean']:.4f}, SD={grand_means['dp_sd']:.4f}, Precision Uncertainty={grand_pu['dp_pu']:.4f}, Bias Uncertainty={dp_bu:.4f}\n")
        f.write(f"ΔPt:            mean={dPt_mean:.4f} Bar\n")
        f.write(f"RPM:            mean={grand_means['rs_mean']:.2f}, SD={grand_means['rs_sd']:.2f}, Precision Uncertainty={grand_pu['rs_pu']:.4f}, Bias Uncertainty={rs_bu:.4f}\n")
        f.write(f"Torque:         mean={grand_means['tq_mean']:.4f}, SD={grand_means['tq_sd']:.4f}, Precision Uncertainty={grand_pu['tq_pu']:.4f}, Bias Uncertainty={tq_bu:.4f}\n")
        
        f.write(f"Uncertainty of RPM = {rs_uncertainty:.4f} RPM,")
        f.write(f"Uncertainty of Mass Flow Rate = {mfr_uncertainty:.4f} Kg/s,")
        f.write(f"Uncertainty of ΔPs (RAW) = {dp_uncertainty:.4f} Bar,")
        f.write(f"Uncertainty of ΔPt = {dPt_uncertainty:.4f} Bar,")
        f.write(f"Uncertainty of Torque = {tq_uncertainty:.4f} N-m,")
        
    

if __name__ == '__main__':
    # 1) Base path (up to “Glycerine\”)
    BASE_DIR = r'C:\Users\Krai\OneDrive - Chulalongkorn University\Electric Pump Senior Project\Experimental Record\Glycerine'
    
    # 2) List the final two-level folders you want to process:
    subdirs = [
        r'29042025\Glycerine_13%_7760_0.187Kg-s',
    ]

    # 3) Iterate through each and run
    for sub in subdirs:
        csv_directory = os.path.join(BASE_DIR, sub)
        print(f"\n=== Processing folder: {csv_directory} ===")
        main(csv_directory)



=== Processing folder: C:\Users\Krai\OneDrive - Chulalongkorn University\Electric Pump Senior Project\Experimental Record\Glycerine\29042025\Glycerine_13%_7760_0.187Kg-s ===

Per-file filtered MEANS:
 - Glycerine_29042025_1221_7760RPM_13%_#3_0.184Kg-s: Mass Flow Rate=0.1832, ΔPs=2.6313, RPM=7759.96, Torque=0.3912
 - Glycerine_29042025_1234_7760RPM_13%_#4_0.184Kg-s: Mass Flow Rate=0.1836, ΔPs=2.6318, RPM=7759.87, Torque=0.3891
 - Glycerine_29042025_1732_7760RPM_13%_#6_0.184Kg-s: Mass Flow Rate=0.1831, ΔPs=2.6193, RPM=7759.92, Torque=0.3963
 - Glycerine_29042025_1756_7760RPM_13%_#7_0.184Kg-s: Mass Flow Rate=0.1836, ΔPs=2.6291, RPM=7759.84, Torque=0.4022
 - Glycerine_29042025_1949_7760RPM_13%_#11_0.183Kg-s: Mass Flow Rate=0.1815, ΔPs=2.6264, RPM=7759.80, Torque=0.3987
 - Glycerine_29042025_2034_7760RPM_13%_#13_0.181Kg-s: Mass Flow Rate=0.1807, ΔPs=2.6362, RPM=7759.84, Torque=0.3842

Grand trimmed summary (of MEANS):
 • Mass Flow Rate: mean=0.1826, SD=0.0012, Precision Uncertainty=0.0013,

In [4]:
 subdirs = [
        r'24042025/Glycerine_8%_6730_0.081Kg-s',
        r'24042025/Glycerine_8%_7760_0.094Kg-s',
        r'23042025/Glycerine_8%_7250_0.094Kg-s',
        r'23042025/Glycerine_85%_6730_0.526Kg-s',
        r'23042025/Glycerine_85%_7760_0.603Kg-s',
        r'22042025/Glycerine_52.5%_6730_0.454Kg-s',
        r'22042025/Glycerine_52.5%_7760_0.521Kg-s',
        r'22042025/Glycerine_85%_7250_0.565Kg-s',
        r'21042025/Glycerine_43%_6730_0.416Kg-s',
        r'21042025/Glycerine_43%_7250_0.447Kg-s',
        r'21042025/Glycerine_43%_7760_0.477Kg-s',
        r'21042025/Glycerine_52.5%_7250_0.487Kg-s',
        r'20042025/Glycerine_31.5%_6730_0.344Kg-s',
        r'20042025/Glycerine_31.5%_7250_0.370Kg-s',
        r'20042025/Glycerine_36.5%_6730_0.378Kg-s',
        r'20042025/Glycerine_36.5%_7250_0.408Kg-s',
        r'19042025/Glycerine_13%_6730_0.160Kg-s',
        r'19042025/Glycerine_13%_7760_0.188Kg-s',
        r'18042025/Glycerine_13%_7250_0.175Kg-s',
        r'18042025/Glycerine_19%_7760_0.268Kg-s',
        r'17042025/Glycerine_19%_6730_0.235Kg-s',
        r'17042025/Glycerine_19%_7250_0.253Kg-s',
        r'17042025/Glycerine_25.6%_6730_0.302Kg-s',
        r'17042025/Glycerine_25.6%_7250_0.328Kg-s',
        r'16042025/Glycerine_21%_6730_0.272Kg-s',
        r'26042025/Glycerine_27.5%_7760_0.354Kg-s',
        r'26042025/Glycerine_21%_7760_0.313Kg-s',
        r'26042025/Glycerine_21%_7250_0.281Kg-s',
        r'28042025/Glycerine_13%_6730_0.160Kg-s',
        r'28042025/Glycerine_21%_7250_0.290Kg-s',
        r'28042025/Glycerine_31.5%_7760_0.395Kg-s',
        r'28042025/Glycerine_36.5_7760_0.437Kg-s',
        r'28042025/Glycerine_52.5%_7760_0.521Kg-s',
        r'29042025/Glycerine_13%_7760_0.187Kg-s',
    ]
 
   BASE_DIR = r'/Users/krai/Library/CloudStorage/OneDrive-ChulalongkornUniversity/Electric Pump Senior Project/Experimental Record/Glycerine'
 
 
 
 #Windows
 
  subdirs = [
        r'24042025\Glycerine_8%_6730_0.081Kg-s',
        r'24042025\Glycerine_8%_7760_0.094Kg-s',
        r'23042025\Glycerine_8%_7250_0.094Kg-s',
        r'23042025\Glycerine_85%_6730_0.526Kg-s',
        r'23042025\Glycerine_85%_7760_0.603Kg-s',
        r'22042025\Glycerine_52.5%_6730_0.454Kg-s',
        r'22042025\Glycerine_52.5%_7760_0.521Kg-s',
        r'22042025\Glycerine_85%_7250_0.565Kg-s',
        r'21042025\Glycerine_43%_6730_0.416Kg-s',
        r'21042025\Glycerine_43%_7250_0.447Kg-s',
        r'21042025\Glycerine_43%_7760_0.477Kg-s',
        r'21042025\Glycerine_52.5%_7250_0.487Kg-s',
        r'20042025\Glycerine_31.5%_6730_0.344Kg-s',
        r'20042025\Glycerine_31.5%_7250_0.370Kg-s',
        r'20042025\Glycerine_36.5%_6730_0.378Kg-s',
        r'20042025\Glycerine_36.5%_7250_0.408Kg-s',
        r'19042025\Glycerine_13%_6730_0.160Kg-s',
        r'19042025\Glycerine_13%_7760_0.188Kg-s',
        r'18042025\Glycerine_13%_7250_0.175Kg-s',
        r'18042025\Glycerine_19%_7760_0.268Kg-s',
        r'17042025\Glycerine_19%_6730_0.235Kg-s',
        r'17042025\Glycerine_19%_7250_0.253Kg-s',
        r'17042025\Glycerine_25.6%_6730_0.302Kg-s',
        r'17042025\Glycerine_25.6%_7250_0.328Kg-s',
        r'16042025\Glycerine_21%_6730_0.272Kg-s',
        r'26042025\Glycerine_27.5%_7760_0.354Kg-s',
        r'26042025\Glycerine_21%_7760_0.313Kg-s',
        r'26042025\Glycerine_21%_7250_0.281Kg-s',
        r'28042025\Glycerine_13%_6730_0.160Kg-s',
        r'28042025\Glycerine_21%_7250_0.290Kg-s',
        r'28042025\Glycerine_31.5%_7760_0.395Kg-s',
        r'28042025\Glycerine_36.5_7760_0.437Kg-s',
        r'28042025\Glycerine_52.5%_7760_0.521Kg-s',
        r'29042025\Glycerine_13%_7760_0.187Kg-s',
    ]

  BASE_DIR = r'C:\Users\Krai\OneDrive - Chulalongkorn University\Electric Pump Senior Project\Experimental Record\Glycerine'

IndentationError: unindent does not match any outer indentation level (<string>, line 44)

In [3]:
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress
import matplotlib.font_manager as font_manager

def process_file(file_path):
    font_legend = {'family': 'TH Sarabun New',
                    'weight': 'bold',
                    'style': 'normal', 
                    'size': 18
                   }
    font_axis = {'family': 'TH Sarabun New',
                    'weight': 'bold',
                    'style': 'normal', 
                    'size': 20
                   }
    base_name   = os.path.splitext(os.path.basename(file_path))[0]
    parent_dir  = os.path.dirname(file_path)
    output_dir  = os.path.join(parent_dir, f"{base_name} plots")
    output_dir  = os.path.join(output_dir, f"combined plots")
    os.makedirs(output_dir, exist_ok=True)

    # Read CSV, skip first two summary rows, use third row as header
    df = pd.read_csv(file_path, delimiter=',', skiprows=2, header=0)

    # --- COMBINED FULL PLOTS ---

    # 1) Torque and Pressure vs Weight
    x = df['Weight [Kg]']
    #y = df['Weight [Kg]']

    fig, ax1 = plt.subplots()
    ax2 = ax1.twinx()
    #plt.plot(x, y, label='Data')
    #plt.xlabel('Run Time [s]')
    #plt.ylabel('Weight [Kg]')
    #plt.title('Mass Flow Rate vs Run Time')
    #plt.legend()
    #plt.ylim(bottom=0)
    #plt.tight_layout()

    lns1 = ax1.plot(x, df['Differential Pressure [Bar]'],label='Differential Pressure')
    ax1.set_xlabel('Weight [Kg]', fontdict=font_axis)
    ax1.set_ylabel('Differential Pressure [Bar]', fontdict=font_axis)
    ax1.set_ylim(bottom=0)

    lns2 = ax2.plot(x, df["Torque [N-m]"],color="orange",label='Torque')
    ax2.set_ylabel('Torque [N-m]', fontdict=font_axis)
    ax2.set_ylim([0,0.6])
    
    lns = lns1 + lns2
    
    labs = [l.get_label() for l in lns]       
    ax1.legend(lns, labs, loc="lower right",prop=font_legend)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f"{base_name}_Pressure_&_Torque_vs_Weight_full_combined_plot.png"))
    plt.close()
    
    # 2) Torque and Pressure vs Time
    x = df['Run Time [s]']
    #y = df['Weight [Kg]']

    fig, ax1 = plt.subplots()
    ax2 = ax1.twinx()
    #plt.plot(x, y, label='Data')
    #plt.xlabel('Run Time [s]')
    #plt.ylabel('Weight [Kg]')
    #plt.title('Mass Flow Rate vs Run Time')
    #plt.legend()
    #plt.ylim(bottom=0)
    #plt.tight_layout()

    lns1 = ax1.plot(x, df['Differential Pressure [Bar]'],label='Differential Pressure')
    ax1.set_xlabel('Run Time [s]', fontdict=font_axis)
    ax1.set_ylabel('Differential Pressure [Bar]', fontdict=font_axis)
    ax1.set_ylim(bottom=0)
    
    lns2 = ax2.plot(x, df["Torque [N-m]"],color="orange",label='Torque')
    ax2.set_ylabel('Torque [N-m]',fontsize = 16, fontdict=font_axis)
    ax2.set_ylim([0,0.6])
    
    lns = lns1 + lns2

    labs = [l.get_label() for l in lns]       
    ax1.legend(lns, labs, loc="lower right",prop=font_legend)
    
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f"{base_name}_Pressure_&_Torque_vs_Run_Time_full_combined_plot.png"))
    plt.close()

def main(csv_dir):
    files = glob.glob(os.path.join(csv_dir, '*.csv'))
    if not files:
        print(f"No CSV files found in {csv_dir}")
        return

    summaries = [process_file(f) for f in files]

    
    
if __name__ == '__main__':
    # 1) Base path (up to “Glycerine\”)
    BASE_DIR = r'C:\Users\Krai\OneDrive - Chulalongkorn University\Electric Pump Senior Project\Experimental Record\Glycerine'
    
    # 2) List the final two-level folders you want to process:
    subdirs = [
        r'24042025\Glycerine_8%_6730_0.081Kg-s',
        r'24042025\Glycerine_8%_7760_0.094Kg-s',
        r'23042025\Glycerine_8%_7250_0.094Kg-s',
        r'23042025\Glycerine_85%_6730_0.526Kg-s',
        r'23042025\Glycerine_85%_7760_0.603Kg-s',
        r'22042025\Glycerine_52.5%_6730_0.454Kg-s',
        r'22042025\Glycerine_52.5%_7760_0.521Kg-s',
        r'22042025\Glycerine_85%_7250_0.565Kg-s',
        r'21042025\Glycerine_43%_6730_0.416Kg-s',
        r'21042025\Glycerine_43%_7250_0.447Kg-s',
        r'21042025\Glycerine_43%_7760_0.477Kg-s',
        r'21042025\Glycerine_52.5%_7250_0.487Kg-s',
        r'20042025\Glycerine_31.5%_6730_0.344Kg-s',
        r'20042025\Glycerine_31.5%_7250_0.370Kg-s',
        r'20042025\Glycerine_36.5%_6730_0.378Kg-s',
        r'20042025\Glycerine_36.5%_7250_0.408Kg-s',
        r'19042025\Glycerine_13%_6730_0.160Kg-s',
        r'19042025\Glycerine_13%_7760_0.188Kg-s',
        r'18042025\Glycerine_13%_7250_0.175Kg-s',
        r'18042025\Glycerine_19%_7760_0.268Kg-s',
        r'17042025\Glycerine_19%_6730_0.235Kg-s',
        r'17042025\Glycerine_19%_7250_0.253Kg-s',
        r'17042025\Glycerine_25.6%_6730_0.302Kg-s',
        r'17042025\Glycerine_25.6%_7250_0.328Kg-s',
        r'16042025\Glycerine_21%_6730_0.272Kg-s',
        r'26042025\Glycerine_27.5%_7760_0.354Kg-s',
        r'26042025\Glycerine_21%_7760_0.313Kg-s',
        r'26042025\Glycerine_21%_7250_0.281Kg-s',
        r'28042025\Glycerine_13%_6730_0.160Kg-s',
        r'28042025\Glycerine_21%_7250_0.290Kg-s',
        r'28042025\Glycerine_31.5%_7760_0.395Kg-s',
        r'28042025\Glycerine_36.5_7760_0.437Kg-s',
        r'28042025\Glycerine_52.5%_7760_0.521Kg-s',
        r'29042025\Glycerine_13%_7760_0.187Kg-s',
    ]

    # 3) Iterate through each and run
    for sub in subdirs:
        csv_directory = os.path.join(BASE_DIR, sub)
        print(f"\n=== Processing folder: {csv_directory} ===")
        main(csv_directory)



=== Processing folder: C:\Users\Krai\OneDrive - Chulalongkorn University\Electric Pump Senior Project\Experimental Record\Glycerine\24042025\Glycerine_8%_6730_0.081Kg-s ===

=== Processing folder: C:\Users\Krai\OneDrive - Chulalongkorn University\Electric Pump Senior Project\Experimental Record\Glycerine\24042025\Glycerine_8%_7760_0.094Kg-s ===

=== Processing folder: C:\Users\Krai\OneDrive - Chulalongkorn University\Electric Pump Senior Project\Experimental Record\Glycerine\23042025\Glycerine_8%_7250_0.094Kg-s ===

=== Processing folder: C:\Users\Krai\OneDrive - Chulalongkorn University\Electric Pump Senior Project\Experimental Record\Glycerine\23042025\Glycerine_85%_6730_0.526Kg-s ===

=== Processing folder: C:\Users\Krai\OneDrive - Chulalongkorn University\Electric Pump Senior Project\Experimental Record\Glycerine\23042025\Glycerine_85%_7760_0.603Kg-s ===

=== Processing folder: C:\Users\Krai\OneDrive - Chulalongkorn University\Electric Pump Senior Project\Experimental Record\Glyce